# Реализация нахождения уравнений движения с помощью Урванений Татаринова для платформы с одним колесом


In [3]:
import sympy as sp
import cloudpickle as pickle

from copy import copy
from sympy import Derivative, Symbol, IndexedBase, Eq, Idx, Sum, Function, Matrix, Add
from sympy import diff, symbols, solve, simplify, poly, pprint, factor, integrate, lambdify
from sympy import cos, sin, pi
from abc import ABC, abstractmethod

In [206]:
def create_fs(self, fs):
    """ not good """
    # nu1, nu2, nu3 = symbols('nu1, nu2, nu3')
    return fs.subs({
        _nu1: Function('nu1_')(t),
        _nu2: Function('nu2_')(t),
        _nu3: Function('nu3_')(t)
    })

def create_ss(self, fs):
    """ not good """
    # nu1, nu2, nu3 = symbols('nu1, nu2, nu3')
    return fs.subs({
        Function('nu1_')(t): _nu1,
        Function('nu2_')(t): _nu2,
        Function('nu3_')(t): _nu3
    })

def save(data, name):
    with open(f'{name}.pickle', 'wb') as f:
        pickle.dump(copy(data), f)
        
def load(name):
    with open(f'{name}.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

def MatrixElDerivative(mat):
    return Matrix([x.diff(t) for x in mat])

def subs_Eqs(eq, Eqs):
    """ 
        Например: x = z+2*y
        Это eq.subs({x: z+2*y})
    """
    for Eq in Eqs:
        left, right = Eq.args[0], Eq.args[1]
        eq = eq.subs({left: right})
    return eq

### Уравнения Татаринова
$$
\frac{d}{dt} \frac{\partial L*}{\partial \omega_{\alpha}} +
\{P_{\alpha}, L*\} = 
\{P_{\alpha}, \sum_{\mu}{\omega_{\mu} P_{\mu}} \} +
\sum_{i}{Q_i \frac{\partial v_i}{ \partial \omega_{\alpha}}}
$$

### Скобка Пуассона
$$
\{f, g\} = \sum_{i=1}^{N} \left( \frac{\partial f}{\partial q_{i}} \frac{\partial g}{\partial p_{i}} - \frac{\partial f}{\partial p_i} \frac{\partial g}{\partial q_i}\right).
$$

### Функции $ P_{\alpha} $
$$
\sum_{\alpha}{P_{\alpha} \omega_{\alpha}} = \sum_i{p_iv_i}
$$

### Функции $v_i$
$$
v_i = \dot{q}
$$

In [5]:
class MechanicalSystem():
    
    def __init__(self):
        pass
    
    def set_q(self, q):
        self.q = q
    
    def set_p(self, p):
        self.p = p
    
    def poisson_bracket(self, F, G): # надо сделать, чтобы использовалось IndexedBase
        """ To evaluate an unevaluated derivative, use the doit method. """
        res = 0
        for i in range(3):
            res += diff(F, self.q[i])*diff(G, self.p[i])
            res -= diff(F, self.p[i])*diff(G, self.q[i])
        return res
    
    @staticmethod
    def sub_Eq(equation, Eq):
        return equation.subs(Eq.args[0], Eq.args[1])
    
    @classmethod
    def sub_Eqs(cls, equation, Eqs):
        sub_equation = equation
        for Eq in Eqs:
            sub_equation = cls.sub_Eq(sub_equation, Eq)
        return sub_equation 
    
    @staticmethod
    def left_part_Eqs(Eqs):
        return [Eq.args[0] for Eq in Eqs]
    
    @staticmethod
    def right_part_Eqs(Eqs):
        return [Eq.args[1] for Eq in Eqs]
    
    def set_constraints(self, constraints):
        """ Setting up constraints """
        self.constraints = constraints
        
    def sub_constraints(self, func):
        """ Substitutes constraints in the equation
        Private special case:
            L -> L* 
        """
        sub_dict = {conn.args[0]: conn.args[1] for conn in self.constraints}
        return func.subs(sub_dict)
    
    def sub_constraints_to_list(self, equations):
        return [self.sub_constraints(equation) for equation in equations]
    
    @staticmethod
    def diff_hack(equation, by):
        """ eq -> eq.subs(Derivative -> temp_variable) -> eq.diff(temp_variable) ->
            -> eq.subs(temp_variable -> Derivative)
        """
        tmp_by = Symbol('tmp_by')
        tmp_eq = equation.subs({by: tmp_by})
        tmp_eq = tmp_eq.diff(tmp_by)
        return tmp_eq.subs({tmp_by: by})
        

In [41]:
class TatarinovSystem(MechanicalSystem):
    
    def __init__(self, N=3, debug=True):
        super().__init__()
        self.N = N
        self.tatarinov_equations = [None]*N
        self.debug = True
        
    def set_omega_equations(self, omegas, equations):
        omega_equations = [Eq(omega, equation) for omega, equation in zip(omegas, equations)]
        self.omega_equations = omega_equations
        
    def set_v_equations(self, vs, equations): # vs - не лучшее название
        v_equations = [Eq(v, equation) for v, equation in zip(vs, equations)]
        self.v_equations = v_equations

    def set_P(self):
        k, mu = symbols('k, mu', cls=Idx)
        left = Sum(P[k]*om[k], (k, 1, self.N)).doit()
        right = Sum(p[i]*v[i], (k, 1, self.N))
        Eq(left, right)
    
    def set_L(self, L):
        self.L = L
        
    def set_L_star(self, L_star):
        self.L_star = simplify(L_star)
    
    def set_F(self, F):
        self.F = F
    
    def create_r(self):
        r = {}
        r['s'] = x*e['x'] + y*e['y']
        r['p'] = r['s'] + xi*e['xi'] + eta*e['eta']
        self.r = r
        return self.r

    def set_Qs(self, Qs):
        self.Qs = Qs
    
    def create_P(self):
        equations_for_P = Eq(
            Sum(P[k]*_omega[k], (k, 1, S.N)),
            Sum(p[k]*_v[k], (k, 1, S.N)))
        print(equations_for_P)
        # subs omega_i and v_i
        sub_equations_for_P = TatarinovSystem.sub_Eqs(
            equations_for_P.doit(), # important
            self.omega_equations + self.v_equations) # for Ex: omega[1] -> nu1), v[1] -> x', ....
        # subs_constraints
        sub_conn_equations_for_P = self.sub_constraints(sub_equations_for_P)
        print(sub_conn_equations_for_P)
        # to equate the coefficients
        left_coeffs = poly(sub_conn_equations_for_P.args[0], TatarinovSystem.right_part_Eqs(self.omega_equations)).coeffs()
        right_coeffs = poly(sub_conn_equations_for_P.args[1], TatarinovSystem.right_part_Eqs(self.omega_equations)).coeffs()
        final_equations_for_P = [Eq(left, right) for left, right in zip(left_coeffs, right_coeffs)]
        self.Ps = final_equations_for_P
        return final_equations_for_P
    
    def create_Q(self, F, r_p):
        self.Q = [F.dot(r_p.diff(x)) for x in self.q]
        
    def set_Q(self, Q):
        self.Q = Q
    
    def Q_dw_by_dv(self, i):
        Q_dw_by_dv = lambda i,a: self.Q[i]*Derivative(self.right_part_Eqs(S.sub_constraints_to_list(S.v_equations))[i],
                                                      self.right_part_Eqs(self.omega_equations)[a]).doit()
        Q_dw_by_dv_sum = lambda a: Q_dw_by_dv(0,a) + Q_dw_by_dv(1,a) + Q_dw_by_dv(2,a) + Q_dw_by_dv(3,a) + Q_dw_by_dv(4,a)
        return Q_dw_by_dv_sum(i)
    
    def tatarinov_equation__depricated(self, i):
        left, right = 0, 0;
        left += diff(diff(L, omega[k]), t)
        left += poisson_bracket(P[k], L)
        right += poisson_bracket(P[k], Sum(omega[k]*P[k], (k, 1, self.N)))
        right += Sum(Q[l]*diff(v[l], omega[k]), (l, 1, self.N))
        Eq(left, right)
        
    def create_bracket_sum(self):
        """ not good """
        # return  _nu1*(p[1]*cos(alpha) + p[2]*sin(alpha)) + _nu2*(-p[1]*sin(alpha) + p[2]*cos(alpha)) + _nu3*p[3]
        bracket_sum = [x*y for x,y in zip(S.right_part_Eqs(S.Ps), S.right_part_Eqs(S.omega_equations))]
        res = 0
        for x in range(3):
            res += bracket_sum[x]
        return res
    
    def solve_tatarinov_equations(self):
        te = [self.tatarinov_equations[i] for x in range(3)]
        sol = solve(te, [Derivative(self.create_fs(x), t) for x in S.right_part_Eqs(S.omega_equations)])
        return sol

    def tatarinov_equation(self, i):
        left, right = 0,0
        left = self.L_star.args[1]
#         left = self.create_fs(left.diff(self.right_part_Eqs(self.omega_equations)[i])).diff(t)
        left = left.diff(self.right_part_Eqs(self.omega_equations)[i])
        left = simplify(left)
        left = left.diff(t)
        left = simplify(self.sub_constraints(left))
        tmp = self.poisson_bracket(self.Ps[i].args[1], self.L_star.args[1])
        left += simplify(self.sub_constraints(tmp))
#         left = self.create_fs(left)
        if self.debug:
            display(left)
        
        right += self.poisson_bracket(self.Ps[i].args[1], self.create_bracket_sum())
        right += simplify(self.Q_dw_by_dv(i))
#         right = self.create_fs(right)
        if self.debug:
            display(right)
        
        ps = lambda i: self.sub_constraints(self.diff_hack(self.L.args[1], self.right_part_Eqs(self.v_equations)[i]))
        self.tatarinov_equations[i] = Eq(left, right).subs({p[1]: ps(0), p[2]: ps(1), p[3]: ps(2)})
        
#         self.tatarinov_equations[i] = self.create_fs(simplify(self.tatarinov_equations[i]))
        self.tatarinov_equations[i] = simplify(self.tatarinov_equations[i])
        
        return self.tatarinov_equations[i]

    def display_tatarinov_equations(self):
        for eq in self.tatarinov_equations:
            display(eq)

##  Импорт платформы

In [217]:
%load_ext autoreload
%autoreload 2

from variables import *
from structure import *
from functions import *

from latex import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Псевдоскорости $nu_1$, $nu_2$
#### Координаты $\theta_1$, $\psi_1$, $\alpha$
#### Связи $\psi_1(\nu_1,\nu_2), \theta_1(\nu_1,\nu_2)$

#### Скорости

In [9]:
omega['platform']= lambda i: Derivative(alpha,t)*e['z']
omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)

In [10]:
v['S'] = euler(S, P)
v['P'] = euler(P, C)
v['C'] = euler(C, D)
v['D'] = lambda i: Matrix([0,0,0]) # проскальзывания нет

In [11]:
v['S'](0)

Matrix([
[ -d*(sin(alpha(t))*cos(theta0(t)) + sin(theta0(t))*cos(alpha(t)))*(Derivative(alpha(t), t) + Derivative(theta0(t), t)) - r*(-sin(alpha(t))*sin(theta0(t)) + cos(alpha(t))*cos(theta0(t)))*Derivative(psi0(t), t) - (sin(beta0)*cos(alpha(t)) + sin(alpha(t))*cos(beta0))*Derivative(alpha(t), t)],
[d*(-sin(alpha(t))*sin(theta0(t)) + cos(alpha(t))*cos(theta0(t)))*(Derivative(alpha(t), t) + Derivative(theta0(t), t)) + r*(-sin(alpha(t))*cos(theta0(t)) - sin(theta0(t))*cos(alpha(t)))*Derivative(psi0(t), t) + (-sin(beta0)*sin(alpha(t)) + cos(beta0)*cos(alpha(t)))*Derivative(alpha(t), t)],
[                                                                                                                                                                                                                                                                                               0]])

#### Угловые скорости

In [12]:
omega['platform']= lambda i: Derivative(alpha,t)*e['z']
omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)

In [218]:
eq['nu1'] = lambda i: scalar(v['S'](i), e['xi'])
eq['nu2'] = lambda i: scalar(v['S'](i), e['eta'])

#### Псевдоскорости

In [14]:
simplify(Matrix([Eq(nu[1], eq['nu1'](0)),
                 Eq(nu[2], eq['nu2'](0))]))

Matrix([
[Eq(nu1(t), -d*sin(theta0(t))*Derivative(alpha(t), t) - d*sin(theta0(t))*Derivative(theta0(t), t) - r*cos(theta0(t))*Derivative(psi0(t), t) - sin(beta0)*Derivative(alpha(t), t))],
[ Eq(nu2(t), d*cos(theta0(t))*Derivative(alpha(t), t) + d*cos(theta0(t))*Derivative(theta0(t), t) - r*sin(theta0(t))*Derivative(psi0(t), t) + cos(beta0)*Derivative(alpha(t), t))]])

In [15]:
eq['f(nu1,nu2)'] = lambda i: solve(
                      [Eq(eq['nu1'](i), nu[1]), Eq(eq['nu2'](i), nu[2])],
                      [Derivative(psi[i],t), Derivative(theta[i],t)],
                      dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)

In [16]:
Eq(Derivative(alpha), nu[3])

Eq(Derivative(alpha(t), t), nu3(t))

####  Связи

In [17]:
eq['diff(psi)_nu']   = lambda i: eq['f(nu1,nu2)'](i)[Derivative(psi[i],t)]
eq['diff(theta)_nu'] = lambda i: eq['f(nu1,nu2)'](i)[Derivative(theta[i],t)]

In [18]:
Matrix([Eq(Derivative(psi[0], t), eq['diff(psi)_nu'](0)),
        Eq(Derivative(theta[0], t), eq['diff(theta)_nu'](0))])

Matrix([
[                              Eq(Derivative(psi0(t), t), -(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + sin(beta0 - theta0(t))*Derivative(alpha(t), t))/r)],
[Eq(Derivative(theta0(t), t), (-d*Derivative(alpha(t), t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - cos(beta0 - theta0(t))*Derivative(alpha(t), t))/d)]])

In [19]:
# L, m, x, y, t, alpha = symbols('L, m, x, y, t, alpha')
# _nu1, _nu2, _nu3, _nu4, _nu5 = symbols('nu1_, nu2_, nu3_, nu4_, nu5_')
# W, T = symbols('W, T')
# s = symbols('s')
P = IndexedBase('P')
p = IndexedBase('p')
# p = IndexedBase('p')
_omega = IndexedBase('omega')
_v = IndexedBase('v')
# Q = IndexedBase('Q')
k = symbols('k', cls=Idx)
# J = IndexedBase('J')

# xi, eta = symbols('xi, eta')
# e = {}
# e['x'] = Matrix([1,0])
# e['y'] = Matrix([0,1])
# e['xi'] = Matrix([cos(alpha), sin(alpha)])
# e['eta'] = Matrix([-sin(alpha), cos(alpha)])

In [20]:
# A, C, c = symbols('A, C, c')

# psi, theta, phi = symbols('psi, theta, phi') # v

# g = symbols('g')

$$ L = T - V $$
$$ T = T_{pl} + T_{fo} + T_{wh} $$

$$ T_{pl} = \frac{m_{pl} v_S^2}{2} + \frac{J_{pl} \omega_{pl}^2}{2} $$
$$ T_{fo} = 0$$
$$ T_{wh} = \frac{m_{wh} v_S^2}{2} + \frac{J_{wh} \omega_{wh}^2}{2} $$

#### Platform

In [21]:
m['platform']

m1

In [22]:
Derivative(x, t)**2 + Derivative(y, t)**2

Derivative(x(t), t)**2 + Derivative(y(t), t)**2

In [23]:
J['platform'] = eye(3,3)*a

In [24]:
omega['platform'](0)

Matrix([
[                      0],
[                      0],
[Derivative(alpha(t), t)]])

#### Wheel

In [25]:
m['wheel']

m2

In [ ]:
Matrix([x,y,0]) + d*e_wheel(0)

In [27]:
J['wheel'] = Matrix([[b,0,0],[0,c,0],[0,0,b]])

#### В одинаковых базисах нужно записывать угловую скорость и момент инерции !

In [28]:
omega['wheel_in'] = lambda i: psi[i].diff(t)*e['y'] + (theta[i].diff(t) + alpha.diff(t))*e['z']

In [ ]:
omega['wheel_in'](0)

### Моменты сил
####  (platform) $ \vec{M} = -W_0*\vec{e}_z $ 
####  (fork) $ \vec{M} = W_0*\vec{e}_z - T_0*\vec{n}_0 $ 
####  (wheel) $ \vec{M} = T_0*\vec{n}_0 $ 

In [219]:
eq['nu1(x,y)'] = Eq(nu[1], Derivative(x,t)*cos(alpha) + Derivative(y,t)*sin(alpha))
eq['nu2(x,y)'] = Eq(nu[2], -Derivative(x,t)*sin(alpha) + Derivative(y,t)*cos(alpha))

In [31]:
eq['diff_x(nu1,nu2)'], eq['diff_y(nu1,nu2)'] = linsolve([eq['nu1(x,y)'], eq['nu2(x,y)']], (Derivative(x, t), Derivative(y, t))).args[0]

In [32]:
def subs_xy(obj):
    substed = obj.subs({nu[1]: eq['nu1(x,y)'],
                        nu[2]: eq['nu2(x,y)']})
    return substed

### L

In [33]:
L = Symbol('L')

In [34]:
pl = m['platform']/2*(Derivative(x, t)**2 + Derivative(y, t)**2)
pl += (J['platform']*omega['platform'](0)).dot(omega['platform'](0))/2

wh = m['wheel']/2*(MatrixElDerivative(Matrix([x,y,0]) + d*e_wheel(0))).dot(MatrixElDerivative(Matrix([x,y,0]) + d*e_wheel(0)))
wh += (J['wheel']*omega['wheel_in'](0)).dot(omega['wheel_in'](0))/2

In [ ]:
L = Eq(L, pl + wh)
L

In [36]:
%%time
sL = simplify(L)

CPU times: user 14.3 s, sys: 102 ms, total: 14.4 s
Wall time: 14.6 s


In [37]:
sL

Eq(L, a*Derivative(alpha(t), t)**2/2 + b*(Derivative(alpha(t), t) + Derivative(theta0(t), t))**2/2 + c*Derivative(psi0(t), t)**2/2 + m1*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)/2 + m2*((d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*sin(alpha(t) + theta0(t)) - Derivative(x(t), t))**2 + (d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*cos(alpha(t) + theta0(t)) + Derivative(y(t), t))**2)/2)

### Определим связи 

In [38]:
conn1 = Eq(Derivative(x, t), eq['diff_x(nu1,nu2)'])
conn2 = Eq(Derivative(y, t), eq['diff_y(nu1,nu2)'])
conn3 = Eq(Derivative(alpha, t), nu[3])
conn4 = Eq(Derivative(theta[0], t), simplify(eq['diff(theta)_nu'](0))).subs({Derivative(alpha, t): nu[3]})
conn5 = Eq(Derivative(psi[0], t), simplify(eq['diff(psi)_nu'](0))).subs({Derivative(alpha, t): nu[3]})

In [39]:
for conn in [conn1, conn2, conn3, conn4, conn5]:
    display(conn)

Eq(Derivative(x(t), t), nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t)))

Eq(Derivative(y(t), t), nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t)))

Eq(Derivative(alpha(t), t), nu3(t))

Eq(Derivative(theta0(t), t), (-d*nu3(t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))/d)

Eq(Derivative(psi0(t), t), -(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))/r)

## Заполним что нужно (связи, уравнения для $\omega_i$, $v_i$, $F$)

### Создадим объект для механической системы с уравнениями Татаринова 

In [220]:
S = TatarinovSystem()

In [221]:
S.set_L(sL)

In [222]:
S.set_constraints([conn1, conn2, conn3, conn4, conn5])
S.constraints

[Eq(Derivative(x(t), t), nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t))),
 Eq(Derivative(y(t), t), nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t))),
 Eq(Derivative(alpha(t), t), nu3(t)),
 Eq(Derivative(theta0(t), t), (-d*nu3(t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))/d),
 Eq(Derivative(psi0(t), t), -(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))/r)]

In [ ]:
%%time
sL_star = S.set_L_star(S.sub_constraints(sL))

In [ ]:
S.set_p([p[1], p[2], p[3]]) # это скорее внутренняя переменная и её не надо задавать here 
S.set_q([x, y, alpha, theta[0], psi[0]])

In [ ]:
S.set_omega_equations(omegas=[_omega[1], _omega[2], _omega[3]],
                      equations=[nu[1], nu[2], nu[3]])
S.omega_equations

In [ ]:
S.set_v_equations(vs=[_v[1], _v[2], _v[3], _v[4], _v[5]],
                  equations=[Derivative(x, t), Derivative(y, t), Derivative(alpha, t), Derivative(theta[0], t), Derivative(psi[0], t)])
S.v_equations

### Создадим $P_{\alpha}$

In [ ]:
Ps = S.create_P()
Ps

### Создадим $Q_i$

In [50]:
# F = Matrix([0,0,0])
# S.set_F(F)

# S.create_Q(S.F, S.create_r()['p'])
# S.Q

S.set_Q([0,0,0,W[1],T[1]])

In [51]:
for i in range(3):
    display(S.Q_dw_by_dv(i))

-T1(t)*cos(theta0(t))/r - W1(t)*sin(theta0(t))/d

-T1(t)*sin(theta0(t))/r + W1(t)*cos(theta0(t))/d

-T1(t)*sin(beta0 - theta0(t))/r + (-d - cos(beta0 - theta0(t)))*W1(t)/d

### Получаем уравнения

##### Проверим что функция работает верно

In [ ]:
i = 0

In [ ]:
left, right = 0,0

In [ ]:
left = S.L_star.args[1]

In [ ]:
tmp = S.right_part_Eqs(S.omega_equations)[i]
tmp

In [ ]:
tmp = left.diff(tmp)
tmp

In [ ]:
%%time
tmp = simplify(tmp.diff(t))

In [ ]:
tmp

In [ ]:
left = tmp

In [ ]:
S.L

In [ ]:
simplify(S.L_star.args[1].subs({alpha: 8}) - S.L_star.args[1]) # !

In [ ]:
S.Ps[i].args[1]

In [ ]:
simplify(S.L_star.args[1])

In [ ]:
tmp = S.poisson_bracket(S.Ps[i].args[1], S.L_star.args[1])

In [ ]:
left += tmp

In [ ]:
simplify(S.sub_constraints(left))

In [ ]:
right

In [ ]:
S.Ps[i].args[1]

In [ ]:
S.create_bracket_sum()

In [ ]:
S.poisson_bracket(S.Ps[i].args[1], S.create_bracket_sum())

In [ ]:
right += S.poisson_bracket(S.Ps[i].args[1], S.create_bracket_sum())

In [ ]:
right

In [ ]:
S.Q_dw_by_dv(i)

In [ ]:
right += simplify(S.Q_dw_by_dv(i))

In [ ]:
S.L.args[1]

In [ ]:
S.right_part_Eqs(S.v_equations)[i]

In [ ]:
S.sub_constraints(S.diff_hack(S.L.args[1], S.right_part_Eqs(S.v_equations)[i]))

In [ ]:
simplify(S.sub_constraints(S.diff_hack(S.L.args[1], S.right_part_Eqs(S.v_equations)[i])))

In [ ]:
ps = lambda i: simplify(S.sub_constraints(S.diff_hack(S.L.args[1], S.right_part_Eqs(S.v_equations)[i])))

In [ ]:
ps(0)

In [ ]:
ps(1)

In [ ]:
ps(2)

In [ ]:
ps(3)

In [ ]:
S.v_equations

In [ ]:
Eq(left, right)

In [ ]:
Eq(left, right).subs({p[1]: ps(0), p[2]: ps(1), p[3]: ps(2)})

# ...

## Equations

In [ ]:
%%time 
S.tatarinov_equation(0)

In [ ]:
%%time
S.tatarinov_equation(1)

In [ ]:
%%time
S.tatarinov_equation(2)

In [ ]:
save(S.tatarinov_equations, 'eqs+TW')

## Subs params & Load

In [52]:
eqs = load('eqs+TW')

In [56]:
def par_subs(eq):
    return eq.subs({
        beta[0]: 0,
        d: 2,
        r: 1,
        b: 2,
        c: 1,
        m['platform']: 3,
        m['wheel']: 1
    })

In [ ]:
# simplify(par_subs(eqs[0]).subs(
#     {Derivative(x, t):0 for x in S.right_part_Eqs(S.omega_equations)}
# ))

# par_subs(eqs[0])

### Solve by $T, W$

In [73]:
%%time
resTW = linsolve([par_subs(eq) for eq in eqs[:2]],
                 [T[1], W[1]])

CPU times: user 16.5 s, sys: 39.7 ms, total: 16.5 s
Wall time: 16.6 s


In [144]:
eq['T'] = resTW.args[0][0]
eq['W'] = resTW.args[0][1]

In [259]:
class SolveByTW:
    def __init__(self, T, W, x, y, nu1, nu2):
        self.T = T
        self.W = W
        self.x = x
        self.y = y
        self.nu1 = self._subs_xy(nu1).doit()
        self.nu2 = self._subs_xy(nu2).doit()
    
    def _solve_by_TW(self):
        pass
    
    def set_xy(self, x_eq, y_eq):
        pass
    
    def _subs_xy(self, eq):
        return eq.subs({
            x: self.x,
            y: self.y
        })
    
    def _subs_nu(self, eq):
        return eq.subs({
            nu[1]: self.nu1,
            nu[2]: self.nu2
        })
    
    def _subs_derivative_nu(self, eq):
        return eq.subs({
            Derivative(nu[1],t): self.nu1.diff(t),
            Derivative(nu[2],t): self.nu2.diff(t)
        })
    
    def _subs_all(self, eq):
        return self._subs_conn(self._subs_nu(self._subs_derivative_nu(eq)))
    
    def _subs_conn(self, eq):
        return S.sub_constraints(eq)
    
    def solve(self):
        self.res = [
            simplify(Eq(T[1], self._subs_all(self.T))),
            simplify(Eq(W[1], self._subs_all(self.W)))
        ]
    
    def get_res(self):
        return self.res

In [224]:
STW = SolveByTW(
    T=eq['T'],
    W=eq['W'],
    x=sin(t),
    y=cos(t),
    nu1=eq['nu1(x,y)'].args[1],
    nu2=eq['nu2(x,y)'].args[1],
)

STW.solve()

In [187]:
# for _eq in STW.get_res():
#     display(_eq)

###### *Save*

In [191]:
# save(STW.get_res(), 'solve_by_TW')

### SolveByNuTheta

In [281]:
class SolveByNuTheta(SolveByTW):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def solve(self):
        super().solve()
        
        # nu3' = f(nu3, alpha, theta, t)
        nu3_eq = par_subs(eqs[2])
        nu3_eq = self._subs_all(nu3_eq)
        nu3_eq = solve(nu3_eq, Derivative(nu[3], t))
        nu3_eq = Eq(Derivative(nu[3], t), nu3_eq[0])
        
        # Theta' = g(nu3, alpha, theta, t)
        theta_eq = self._subs_nu(S.constraints[3])
        theta_eq = par_subs(theta_eq)
        
        # alpha' = r(nu3)
        alpha_eq = S.constraints[2]
        
        self.res = [simplify(nu3_eq),
                    simplify(theta_eq),
                    alpha_eq]

In [282]:
%%time
SNT = SolveByNuTheta(
    T=eq['T'],
    W=eq['W'],
    x=sin(t),
    y=cos(t),
    nu1=eq['nu1(x,y)'].args[1],
    nu2=eq['nu2(x,y)'].args[1],
)

SNT.solve()

CPU times: user 1min 39s, sys: 326 ms, total: 1min 39s
Wall time: 1min 42s


In [283]:
for _eq in SNT.get_res():
    display(_eq)

Eq(Derivative(nu3(t), t), (8*T1(t)*sin(theta0(t)) - 4*W1(t)*cos(theta0(t)) - 8*W1(t) - nu3(t)**2*sin(theta0(t)) - 4*nu3(t)**2*sin(2*theta0(t)) - nu3(t)**2*sin(3*theta0(t)) - 10*nu3(t)*cos(t + alpha(t)) - nu3(t)*cos(t + alpha(t) - theta0(t)) + 3*nu3(t)*cos(t + alpha(t) + theta0(t)) + 10*nu3(t)*cos(t + alpha(t) + 2*theta0(t)) + 4*nu3(t)*cos(t + alpha(t) + 3*theta0(t)) + 4*sin(2*t + 2*alpha(t) + 2*theta0(t)) + 3*sin(2*t + 2*alpha(t) + 3*theta0(t)) - 3*sin(theta0(t)) - 14*cos(t + alpha(t)) - 6*cos(t + alpha(t) + 2*theta0(t)))/(4*(2*a + cos(theta0(t))**2 + 2)))

Eq(Derivative(theta0(t), t), -nu3(t)*cos(theta0(t))/2 - nu3(t) - sin(t + alpha(t) + theta0(t))/2)

Eq(Derivative(alpha(t), t), nu3(t))

###### *Save*

In [284]:
save(SNT, 'SNT')

### Integrate above system

In [ ]:
import numpy as np
from scipy.integrate import RK45, odeint

In [285]:
_vars = S.left_part_Eqs(SNT.get_res())
_vars

[Derivative(nu3(t), t), Derivative(theta0(t), t), Derivative(alpha(t), t)]

In [287]:
_eqs = S.right_part_Eqs(SNT.get_res())
_eqs

[(8*T1(t)*sin(theta0(t)) - 4*W1(t)*cos(theta0(t)) - 8*W1(t) - nu3(t)**2*sin(theta0(t)) - 4*nu3(t)**2*sin(2*theta0(t)) - nu3(t)**2*sin(3*theta0(t)) - 10*nu3(t)*cos(t + alpha(t)) - nu3(t)*cos(t + alpha(t) - theta0(t)) + 3*nu3(t)*cos(t + alpha(t) + theta0(t)) + 10*nu3(t)*cos(t + alpha(t) + 2*theta0(t)) + 4*nu3(t)*cos(t + alpha(t) + 3*theta0(t)) + 4*sin(2*t + 2*alpha(t) + 2*theta0(t)) + 3*sin(2*t + 2*alpha(t) + 3*theta0(t)) - 3*sin(theta0(t)) - 14*cos(t + alpha(t)) - 6*cos(t + alpha(t) + 2*theta0(t)))/(4*(2*a + cos(theta0(t))**2 + 2)),
 -nu3(t)*cos(theta0(t))/2 - nu3(t) - sin(t + alpha(t) + theta0(t))/2,
 nu3(t)]

In [ ]:
def f(_vars_values, time):
    return [lambdify(_vars, t, 'numpy')(*_vars_values) for x in eqs]

### Solve by $\dot{\nu_1}, \dot{\nu_2}, \dot{\nu_3}$

In [ ]:
%%time
res = linsolve([par_subs(eq) for eq in eqs],
               [Derivative(x, t) for x in S.right_part_Eqs(S.omega_equations)])
# or 
# res = load('res+TW')

In [ ]:
%%time
s_eq1 = simplify(res.args[0][0])

In [ ]:
%%time
s_eq2 = simplify(res.args[0][1])

In [ ]:
%%time
s_eq3 = simplify(res.args[0][2])

In [ ]:
alpha

In [ ]:
for left_eq, right_eq in zip([s_eq1, s_eq2, s_eq3], [Derivative(x, t) for x in S.right_part_Eqs(S.omega_equations)]):
    display(Eq(left_eq, right_eq))

In [ ]:
for left_eq, right_eq in zip([s_eq1, s_eq2, s_eq3], [Derivative(x, t) for x in S.right_part_Eqs(S.omega_equations)]):
    display(Eq(left_eq, right_eq))

## Найдем  $W,T, \ddot{\alpha}$  как функции от времени 

In [ ]:
from copy import copy

for x in S.tatarinov_equations:
    display(x)

### Зададим $x,y, alpha$ как функции времени

In [ ]:
# Начальные условия и парметры
eq_x = 3*t
eq_y = t**2

params = dict(xi=0, eta=0)

t_end = 2*pi.evalf()

In [ ]:
def _ss(eq):
    nu1, nu2, nu3 = symbols('nu1, nu2, nu3')
    return eq.subs({
        nu1: Function('nu1')(t),
        nu2: Function('nu2')(t),
        nu3: Function('nu3')(t)
    })
    
def subs_funcs(eq):
    subs_dict = dict(x=eq_x, y=eq_y, alpha=Function('alpha')(t)) # , alpha=eq_alpha)
    if type(eq) == dict:
        d = {}
        for k,v in eq.items():
            d[_ss(k)] = _ss(v.subs(subs_dict))
        return d
    if type(eq) == list:
        l = []
        for x in eq:
            l += _ss(eq.subs(subs_dict))
        return l
    if type(eq) == Eq or type(eq) == Add:
        return _ss(eq.subs(subs_dict))

In [ ]:
S.constraints

In [ ]:
nus = solve(S.constraints, S.right_part_Eqs(S.omega_equations))
for k, v in nus.items():
    display(Eq(k, v))

In [ ]:
nust = subs_funcs(nus)
nust

In [ ]:
tet = copy(S.tatarinov_equations)
for x in tet:
    display(x)

In [ ]:
for i in range(3):
    tet[i] = tet[i].subs(nust).doit()
    display(tet[i])
#     display((tet[i].subs(nust)))
#     display((tet[i].subs(nust)).doit())
    

In [ ]:
sol = solve(tet, [T, W, Derivative(Function('alpha')(t), t, 2)])
for k,v in sol.items():
    display(Eq(k, v))

### Проинтегрируем
$$ {\ddot{\alpha}, \dot{\alpha}} \rightarrow \alpha $$

In [ ]:
_vars = [Function('alpha2')(t), Function('alpha')(t)]
_vars

In [ ]:
eqs = [Function('alpha2')(t), list(sol.values())[2]]
eqs

In [ ]:
_vars = [Symbol('alpha'),  Symbol('alpha2')]
eqs =   [Symbol('alpha2'), list(sol.values())[2].subs({Function('alpha')(t): Symbol('alpha')})]

print("Система уравнений на alpha")
for l, r in zip(_vars, eqs):
    display(Eq(l,r))

In [ ]:
# keyword can't be an expression
eqs[1] = eqs[1].subs({J[s]: 3})

_subs = dict(m=1, eta=2, xi=3)

In [ ]:
def f(_vars_values, time):
    return [lambdify(_vars, x.subs(_subs), 'numpy')(*_vars_values) for x in eqs]

In [ ]:
import numpy as np
from scipy.integrate import RK45, odeint

In [ ]:
f0 = [1, 0.1] # [alpha', alpha]
time = np.linspace(0., 4, 100)

_solution = odeint(f, f0, time)

In [ ]:
import matplotlib.pyplot as plt

_items = zip(['dot_alpha', 'alpha'], list(range(2)))

for name, col in _items:
    print(name)
    fig = plt.plot(time, _solution[:, col], label=name)
    plt.show()

## Подставим в $W, T$
$$ \{W(\alpha, t), T(\alpha, t)\} \rightarrow \{W(t), T(t)\}$$

In [ ]:
display(sol[W])
display(sol[T])

In [ ]:
W_t = lambdify([t, Function('alpha')(t)], sol[W].subs(_subs), 'numpy')
T_t = lambdify([t, Function('alpha')(t)], sol[T].subs(_subs), 'numpy')

In [ ]:
_solution[:,1].size == time.size 

In [ ]:
wl, tl = [], []

for x,y in zip(time, _solution[:,1]):
    wl.append(W_t(x, y))
    
for x,y in zip(time, _solution[:,1]):
    tl.append(T_t(x, y))

In [ ]:
fig = plt.plot(time, wl, label='W')
plt.show()

In [ ]:
fig = plt.plot(time, tl, label='T')
plt.show()

In [ ]:
al = _solution[:, 1]

In [ ]:
tmp_x = tl*np.sin(al)*(-1) + wl*np.cos(al)
tmp_y = tl*np.cos(al) + wl*np.sin(al)

In [ ]:
fig = plt.plot(tmp_x, label='~x"')
fig = plt.plot(tmp_y, label='~y"')
plt.show()

In [ ]:
F

## Проинтегрируем численно

In [ ]:
left = []; right = []

In [ ]:
left += S.left_part_Eqs(S.constraints)
right += S.right_part_Eqs(S.constraints)

In [ ]:
diff_omegas_t = [Derivative(S.create_fs(x)) for x in S.right_part_Eqs(S.omega_equations)]
constr = solve(eqs, diff_omegas_t)
constr

In [ ]:
for left_item, right_item in constr.items():
    left += [left_item]
    right += [right_item]

In [ ]:
_vars = (phi, psi, theta, nu1, nu2, nu3)

In [ ]:
right = [S.create_ss(x) for x in right]

In [ ]:
_subs = {A: 1, C: 2, g: 10, m: 1}

In [ ]:
left

In [ ]:
right

In [ ]:
def f(_vars_values, time):
    return [lambdify(_vars, x.subs(_subs), 'numpy')(*_vars_values) for x in right]

In [ ]:
f0 = [3,0.1,0.1,1,2,3]

In [ ]:
import numpy as np
from scipy.integrate import RK45, odeint

In [ ]:
_solution = odeint(f, f0, np.linspace(0., 4., 100))

In [ ]:
import matplotlib.pyplot as plt 

_items = zip(['phi', 'psi', 'theta', 'nu1', 'nu2', 'nu3'], list(range(6)))

for name, col in _items:
    print(name)
    fig = plt.plot(np.linspace(0,4,100), _solution[:, col], label=name)
    plt.show()

## Посмотрим на энергию

In [ ]:
H = A*(Derivative(theta,t)**2 + Derivative(psi,t)**2*sin(theta)**2)/2 \
    + C*(Derivative(phi, t) + Derivative(psi, t)*cos(theta))**2/2 \
    + m*g*cos(theta)
H

In [ ]:
H = S.sub_constraints(H)
H

In [ ]:
fH = lambdify(_vars, H.subs(_subs), 'numpy')

In [ ]:
H_values = np.array([fH(*x) for x in _solution])
max(H_values) - min(H_values)

In [ ]:
for x in S.constraints:
    display(x)

In [ ]:
L